In [1]:
# install packages

In [2]:
pip install sec_api

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install regex

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
import pandas as pd
from sec_api import ExtractorApi
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline

In [5]:
# Pull all N-CSRs from 2021 to present that contain "Russia"
# remove '#' before 'print' to reveal output

In [6]:
from sec_api import FullTextSearchApi

fullTextSearchApi = FullTextSearchApi(api_key="185feb637d04711d886c35f0d5b603b7eb667fbeaae90c903301f213e06a17c0")
# %%
query = {
  "query": '"Russia"',
  "formTypes": ['N-CSR'],
  "startDate": '2021-01-01',
  "endDate": '2022-04-06',
}
# %%
filingsRussia = fullTextSearchApi.get_filings(query)
# %%
#print(filingsRussia)
# %%

In [7]:
# Test to pull a single N-CSR's url
print(filingsRussia['filings'][0]['filingUrl'])

https://www.sec.gov/Archives/edgar/data/860489/000008805321000015/ar103120cee.htm


In [8]:
# A loop to pull key characteristics of all relevant N-CSRs


In [9]:

for i in filingsRussia['filings']:
    print(i['filingUrl'])
    print(i['ticker'])
    print(i['cik'])
    print(i['filedAt'])
    print(i['companyNameLong'])

https://www.sec.gov/Archives/edgar/data/860489/000008805321000015/ar103120cee.htm
CEE
860489
2021-01-08
CENTRAL & EASTERN EUROPE FUND, INC. (CEE) (CIK 0000860489)
https://www.sec.gov/Archives/edgar/data/878087/000186842022000052/primary-document.htm
None
878087
2022-02-28
TEMPLETON DEVELOPING MARKETS TRUST (CIK 0000878087)
https://www.sec.gov/Archives/edgar/data/1137360/000093041321000564/c101237_ex99-906cert.htm
None
1137360
2021-03-10
VanEck Vectors ETF Trust (CIK 0001137360)
https://www.sec.gov/Archives/edgar/data/1137360/000093041322000460/c103163_ncsr.htm
None
1137360
2022-03-10
VanEck ETF Trust (CIK 0001137360)
https://www.sec.gov/Archives/edgar/data/1137360/000093041322000460/c103163_ex99-906cert.htm
None
1137360
2022-03-10
VanEck ETF Trust (CIK 0001137360)
https://www.sec.gov/Archives/edgar/data/1137360/000093041321000564/c101237_ncsr.htm
None
1137360
2021-03-10
VanEck Vectors ETF Trust (CIK 0001137360)
https://www.sec.gov/Archives/edgar/data/878087/000171773421000071/primary-d

In [10]:
# Print first N-CSR
# remove '#'' before 'print' for output

In [11]:
from sec_api import RenderApi

renderApi = RenderApi(api_key="185feb637d04711d886c35f0d5b603b7eb667fbeaae90c903301f213e06a17c0")

url = "https://www.sec.gov/Archives/edgar/data/860489/000008805321000015/ar103120cee.htm"

filing = renderApi.get_filing(url)

NCSR = filing

#print(NCSR)

In [12]:
# clean text

In [13]:
cleantext = BeautifulSoup(NCSR, 'lxml').text


In [14]:
print(cleantext[1:50])

N-CSR
1
ar103120cee.htm
THE CENTRAL AND EASTERN E


In [15]:
# parse sentences

In [16]:
!pip install nltk

import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/rob/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
# remove '#' before 'a' for to reveal cleantext

In [18]:
from nltk.tokenize import sent_tokenize

a=sent_tokenize(cleantext)
#a

In [19]:
# create a dataframe with cleaned/parsed N-CSR sentences

In [20]:
txt_df = DataFrame(a,columns = ['text'])

#txt_df.head()
print(txt_df)

                                                   text
0     \nN-CSR\n1\nar103120cee.htm\nTHE CENTRAL AND E...
1     REPORT TO STOCKHOLDERS\n\n \n \n\n\n\n\n\n\n\n...
2     Instead, the\nreports will be made available o...
3     If you already elected to receive shareholder ...
4     You may elect to receive\nshareholder reports ...
...                                                 ...
1410  (a)(2)\nCertification pursuant to Rule 30a-2(a...
1411  (b)\nCertification pursuant to Rule 30a-2(b) u...
1412  SIGNATURES\n \nPursuant to the requirements of...
1413  Registrant:\nThe Central and Eastern Europe Fu...
1414  By:\n\n/s/Hepsen Uzcan\nHepsen Uzcan\nPresiden...

[1415 rows x 1 columns]


In [21]:
# use zero shot classification to classify "Russia" relevance to N-CSR

In [22]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [23]:
topic_of_interest = ["Russia"]

In [24]:
text_of_interest = ['txt_df']
# different score when using 'cleantext'

In [25]:
res = classifier(text_of_interest, topic_of_interest)

In [26]:
print(res)

[{'sequence': 'txt_df', 'labels': ['Russia'], 'scores': [0.09547276794910431]}]


In [27]:
# Next we need to score each sentence in the dataframe. 
# Once we do this, we will filter the dataframe to only the high-scoring sentences
# Then we will be able to for patterns

In [28]:
#if res['labels'][0] == 'renewable' and res['scores'][0] > 0.5:
    #candidate_results[0] = candidate_results[0] + 1